In [1]:
from __future__ import print_function

In [2]:
import logging
reload(logging)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s.%(msecs).03d %(name)s %(message)s",
    datefmt='%Y-%m-%dT%H:%M:%S'
)

In [3]:
import pickle

In [4]:
import pandas
import numpy
import Bio
import Bio.Alphabet.IUPAC as IUPAC

In [5]:
import theano
import theano.tensor as T

In [6]:
import seaborn
from matplotlib import pylab

In [8]:
raw_source_data = pandas.read_csv("160912_grocklin_data.txt", delim_whitespace=True)

In [9]:
scramble_data = (
    raw_source_data
    .query("data0 > -4.5")
    .query("designtype == 'scramble'")
    [["name", "run", "sequence", "protease", "ec50"]]
    .groupby(["sequence", "protease"])
    .apply(lambda d: d.sort_values(by="ec50", ascending=False).head(1)))

scramble_data["ec50"] = numpy.clip(scramble_data["ec50"], 0, None)
scramble_data = scramble_data[[ "C" not in s for s in scramble_data["sequence"]]]

In [10]:
source_data = {
    t : scramble_data[scramble_data["protease"] == t].sample(frac=1)
    for t in ("chymo", "tryp")
}

In [11]:
import sklearn.cross_validation as cross_validation

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
import sequence_protease_susceptibility
from sequence_protease_susceptibility import CenterLimitedPSSMModel

In [14]:
import itertools

In [15]:
t_cuts = {"tryp" : "RK", "chymo" : "FYLI"}
targets = ("tryp", "chymo")
mses = []

for target in t_cuts:
    tdata = source_data[target].copy()
    tdata["pred_sequence"] = "ASHM" + tdata["sequence"] + "LEGG"
    tdata["is_rd1"] = tdata["name"].map(lambda n: "_random" in n)
    
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(
        list(tdata["pred_sequence"]), tdata["ec50"].values.copy(), test_size = .20)

    m = CenterLimitedPSSMModel(init_aas = t_cuts[target])
    m.fit(X_train, y_train)

    mses.append(dict(
        data="full",
        target=target,
        init="cut",
        mse_train = m.eval_f("mse", X=X_train, y=y_train),
        mse_test = m.eval_f("mse", X=X_test, y=y_test),
        model=m
    ))
    
    m = CenterLimitedPSSMModel()
    m.fit(X_train, y_train)

    mses.append(dict(
        data="full",
        target=target,
        init="zero",
        mse_train = m.eval_f("mse", X=X_train, y=y_train),
        mse_test = m.eval_f("mse", X=X_test, y=y_test),
        model=m
    ))
    
    X_rd1 = list(tdata.query("is_rd1")["pred_sequence"])
    y_rd1 = tdata.query("is_rd1")["ec50"].values.copy()
    X_non_rd1 = list(tdata.query("not is_rd1")["pred_sequence"])
    y_non_rd1 = tdata.query("not is_rd1")["ec50"].values.copy()
    
    m = CenterLimitedPSSMModel(init_aas = t_cuts[target])
    m.fit(X_rd1, y_rd1)

    mses.append(dict(
        data="rd1",
        target=target,
        init="cut",
        mse_train = m.eval_f("mse", X=X_rd1, y=y_rd1),
        mse_test = m.eval_f("mse", X=X_non_rd1, y=y_non_rd1),
        model=m
    ))
    
    m = CenterLimitedPSSMModel()
    m.fit(X_rd1, y_rd1)

    mses.append(dict(
        data="rd1",
        target=target,
        init="zero",
        mse_train = m.eval_f("mse", X=X_rd1, y=y_rd1),
        mse_test = m.eval_f("mse", X=X_non_rd1, y=y_non_rd1),
        model=m
    ))

ERROR (theano.gof.opt): SeqOptimizer apply <theano.compile.mode.AddDestroyHandler object at 0x1090dae10>
2016-10-21T18:08:26.580 theano.gof.opt SeqOptimizer apply <theano.compile.mode.AddDestroyHandler object at 0x1090dae10>
ERROR (theano.gof.opt): Traceback:
2016-10-21T18:08:26.582 theano.gof.opt Traceback:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/usr/local/lib/python2.7/site-packages/theano/gof/opt.py", line 230, in apply
    sub_prof = optimizer.optimize(fgraph)
  File "/usr/local/lib/python2.7/site-packages/theano/gof/opt.py", line 89, in optimize
    ret = self.apply(fgraph, *args, **kwargs)
  File "/usr/local/lib/python2.7/site-packages/theano/compile/mode.py", line 134, in apply
    fgraph.replace_validate(o, _output_guard(o),
  File "/usr/local/lib/python2.7/site-packages/theano/gof/op.py", line 633, in __call__
    (i, ins, node))
ValueError: Cannot compute test value: input 0 (Elemwise{add,no_inplace}.0) of Op OutputGuard(Elemwise{add,no_inplace}.0)

In [24]:
test_results = pandas.DataFrame.from_records(mses)
test_results

,data,init,model,mse_test,mse_train,target
0,full,cut,"CenterLimitedPSSMModel(flanking_window=4, init...",0.775030945904,0.718858153788,chymo
1,full,zero,"CenterLimitedPSSMModel(flanking_window=4, init...",0.782270280886,0.728009260618,chymo
2,rd1,cut,"CenterLimitedPSSMModel(flanking_window=4, init...",0.979481130067,0.689495070523,chymo
3,rd1,zero,"CenterLimitedPSSMModel(flanking_window=4, init...",0.9906758651,0.677505325325,chymo
4,full,cut,"CenterLimitedPSSMModel(flanking_window=4, init...",0.625081393343,0.577452972273,tryp
5,full,zero,"CenterLimitedPSSMModel(flanking_window=4, init...",0.63157185832,0.583361732206,tryp
6,rd1,cut,"CenterLimitedPSSMModel(flanking_window=4, init...",0.755383629234,0.430481596017,tryp
7,rd1,zero,"CenterLimitedPSSMModel(flanking_window=4, init...",0.763712957821,0.423703655051,tryp


In [25]:
for i, r in test_results.query("init=='zero'").iterrows():
    m = r["model"]
    
    pylab.figure()
    pylab.title(r["target"] + " " + r["data"])
    pylab.plot(
        numpy.arange(20),
        m.fit_coeffs_["seq_weights"],
        "o")
    pylab.xticks(numpy.arange(20), list(IUPAC.IUPACProtein.letters))
    
    pylab.figure()
    seaborn.heatmap(m.fit_coeffs_["weights"], xticklabels=IUPAC.IUPACProtein.letters)

In [26]:
def norm_coeffs(model):
    c = numpy.concatenate((
            model.fit_coeffs_["weights"][:4],
            model.fit_coeffs_["weights"][5:]))
    return c / numpy.linalg.norm(c.ravel())

for t in targets:
    models = test_results.query("target=='%s'" % t).set_index(["target", "data", "init"])

    pair_dot = pandas.DataFrame(index=models.index, columns=models.index, dtype=float)
    for i, j in itertools.product(models.index, models.index):
        ic = norm_coeffs(models.ix[i]["model"]).ravel()
        jc = norm_coeffs(models.ix[j]["model"]).ravel()

        pair_dot.ix[i][j] = numpy.abs(numpy.dot(ic.ravel(), jc.ravel()))

    print("Coeff dot product:")
    print(pair_dot.astype(float))
    print()
    
    pylab.figure()
    seaborn.heatmap(pair_dot, vmin=.5)

Coeff dot product:
target                tryp                              
data                  full                 rd1          
init                   cut      zero       cut      zero
target data init                                        
tryp   full cut   1.000000  0.988773  0.848438  0.840673
            zero  0.988773  1.000000  0.864817  0.848835
       rd1  cut   0.848438  0.864817  1.000000  0.983072
            zero  0.840673  0.848835  0.983072  1.000000

Coeff dot product:
target               chymo                              
data                  full                 rd1          
init                   cut      zero       cut      zero
target data init                                        
chymo  full cut   1.000000  0.983451  0.809907  0.799053
            zero  0.983451  1.000000  0.785964  0.770946
       rd1  cut   0.809907  0.785964  1.000000  0.961435
            zero  0.799053  0.770946  0.961435  1.000000



In [27]:
test_results

,data,init,model,mse_test,mse_train,target
0,full,cut,"CenterLimitedPSSMModel(flanking_window=4, init...",0.775030945904,0.718858153788,chymo
1,full,zero,"CenterLimitedPSSMModel(flanking_window=4, init...",0.782270280886,0.728009260618,chymo
2,rd1,cut,"CenterLimitedPSSMModel(flanking_window=4, init...",0.979481130067,0.689495070523,chymo
3,rd1,zero,"CenterLimitedPSSMModel(flanking_window=4, init...",0.9906758651,0.677505325325,chymo
4,full,cut,"CenterLimitedPSSMModel(flanking_window=4, init...",0.625081393343,0.577452972273,tryp
5,full,zero,"CenterLimitedPSSMModel(flanking_window=4, init...",0.63157185832,0.583361732206,tryp
6,rd1,cut,"CenterLimitedPSSMModel(flanking_window=4, init...",0.755383629234,0.430481596017,tryp
7,rd1,zero,"CenterLimitedPSSMModel(flanking_window=4, init...",0.763712957821,0.423703655051,tryp


In [28]:
models = { r["target"] : r["model"] for _, r in test_results.query("init == 'zero'").iterrows() }

In [29]:
results = []
for target, src_data in raw_source_data.query("'X' not in sequence").groupby("protease"):
    src_seq = "ASHM" + src_data.sequence.astype(str) + "LEGG"
    results.append(
        src_seq.groupby(src_seq.map(len))
        .transform(lambda sg: models[target].predict(list(sg)))
        .rename("pred_ec50")
    )
results = pandas.concat(results)

In [30]:
pred_table = raw_source_data.join(results)
pred_table.to_csv("160924_prediction_table.csv", index=False)

In [33]:
ssm_data = pandas.DataFrame.from_dict({
    "sequence" : map(str.strip, open('160924_grocklin_ssm2_myseqs').readlines())
})
ssm_data["pred_sequence"] = "ASHM" + ssm_data.sequence + "LEGG"

In [34]:
for target in models :
    ssm_data["pred_ec50_%s" % target] = (
        ssm_data.pred_sequence.groupby(ssm_data.pred_sequence.map(len))
        .transform(lambda sg: models[target].predict(list(sg)))
        .rename("pred_ec50_%s" % target)
    )

In [35]:
ssm_data.to_csv("160924_ssm2_myseqs_pred_ec50.csv", index=False)

In [36]:
!head 160924_ssm2_myseqs_pred_ec50.csv

sequence,pred_sequence,pred_ec50_chymo,pred_ec50_tryp
TTIKVNGQEYTVPLSPEQAAKAAKKRWPDYEVQIHGNTVKVTR,ASHMTTIKVNGQEYTVPLSPEQAAKAAKKRWPDYEVQIHGNTVKVTRLEGG,1.6451627229435284,2.089522133805952
TTIKVNGQEYTVPLSPEQCAKAAKKRWPDYEVQIHGNTVKVTR,ASHMTTIKVNGQEYTVPLSPEQCAKAAKKRWPDYEVQIHGNTVKVTRLEGG,1.665684556072952,2.1116400034760923
TTIKVNGQEYTVPLSPEQDAKAAKKRWPDYEVQIHGNTVKVTR,ASHMTTIKVNGQEYTVPLSPEQDAKAAKKRWPDYEVQIHGNTVKVTRLEGG,1.880411441748833,2.202999678257806
TTIKVNGQEYTVPLSPEQEAKAAKKRWPDYEVQIHGNTVKVTR,ASHMTTIKVNGQEYTVPLSPEQEAKAAKKRWPDYEVQIHGNTVKVTRLEGG,1.7867565817740654,2.1325417332602656
TTIKVNGQEYTVPLSPEQFAKAAKKRWPDYEVQIHGNTVKVTR,ASHMTTIKVNGQEYTVPLSPEQFAKAAKKRWPDYEVQIHGNTVKVTRLEGG,1.3248208929667444,1.9693770067715213
TTIKVNGQEYTVPLSPEQGAKAAKKRWPDYEVQIHGNTVKVTR,ASHMTTIKVNGQEYTVPLSPEQGAKAAKKRWPDYEVQIHGNTVKVTRLEGG,1.7841550344336465,2.1470199104491337
TTIKVNGQEYTVPLSPEQHAKAAKKRWPDYEVQIHGNTVKVTR,ASHMTTIKVNGQEYTVPLSPEQHAKAAKKRWPDYEVQIHGNTVKVTRLEGG,1.5597293374714614,2.056554721062624
TTIKVNGQEYTVP

In [38]:
from sklearn.cross_validation import cross_val_predict

In [39]:
targets = ("tryp", "chymo")
cv_results = []

for target in targets:
    full_data = source_data[target].copy().sample(frac=1)
    full_data["pred_sequence"] = "ASHM" + full_data["sequence"] + "LEGG"
    full_data["is_rd1"] = full_data["name"].map(lambda n: "_random" in n)
    rd1_data = full_data.query("is_rd1").sample(frac=1)
    
    m = CenterLimitedPSSMModel()
    
    m.fit(list(full_data["pred_sequence"]), full_data["ec50"].values)
    train_full_pred = m.predict(list(full_data["pred_sequence"]))
    cv_full_pred = cross_validation.cross_val_predict(m, list(full_data["pred_sequence"]), full_data["ec50"].values)
    
    cv_results.append(dict(
        target = target,
        data = "full",
        cv_mse = numpy.mean(numpy.square(cv_full_pred - full_data["ec50"])),
        train_mse = numpy.mean(numpy.square(train_full_pred - full_data["ec50"]))
    ))
    
    m.fit(list(rd1_data["pred_sequence"]), rd1_data["ec50"].values)
    train_rd1_pred = m.predict(list(rd1_data["pred_sequence"]))
    cv_rd1_pred = cross_validation.cross_val_predict(m, list(rd1_data["pred_sequence"]), rd1_data["ec50"].values)
    
    cv_results.append(dict(
        target = target,
        data = "rd1",
        cv_mse = numpy.mean(numpy.square(cv_rd1_pred - rd1_data["ec50"])),
        train_mse = numpy.mean(numpy.square(train_rd1_pred - rd1_data["ec50"]))
    ))

cv_results = pandas.DataFrame.from_records(cv_results)

2016-10-21T18:13:58.949 root opt_cycle: 0 vars: ('seq_weights',)
2016-10-21T18:14:03.252 root last_opt iter: 015 mse: 0.849
2016-10-21T18:14:03.253 root opt_cycle: 1 vars: ('ind_w', 'ind_b', 'tot_w', 'tot_b', 'tot_l')
2016-10-21T18:14:07.903 root last_opt iter: 017 mse: 0.833
2016-10-21T18:14:07.904 root opt_cycle: 2 vars: ('weights',)
2016-10-21T18:14:14.077 root last_opt iter: 024 mse: 0.624
2016-10-21T18:14:14.078 root opt_cycle: 3 vars: ('seq_weights',)
2016-10-21T18:14:15.787 root last_opt iter: 005 mse: 0.622
2016-10-21T18:14:15.788 root opt_cycle: 4 vars: ('ind_w', 'ind_b', 'tot_w', 'tot_b', 'tot_l')
2016-10-21T18:14:16.553 root last_opt iter: 001 mse: 0.622
2016-10-21T18:14:16.554 root opt_cycle: 5 vars: ('weights', 'seq_weights')
2016-10-21T18:14:25.437 root last_opt iter: 033 mse: 0.591
2016-10-21T18:14:25.438 root opt_cycle: 6 vars: ('ind_w', 'ind_b', 'tot_w', 'tot_b', 'tot_l')
2016-10-21T18:14:26.889 root last_opt iter: 004 mse: 0.591
2016-10-21T18:14:26.890 root opt_cycle:

In [40]:
cv_results[["target", "data", "train_mse", "cv_mse"]]

,target,data,train_mse,cv_mse
0,tryp,full,0.582827,0.609161
1,tryp,rd1,0.423704,2.115239
2,chymo,full,0.716879,0.766995
3,chymo,rd1,0.677505,1.077561
